# Prerequisites

## Load employees.csv into DataFame

In [0]:
df = spark.read.csv(path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/user_dataset/users_001.csv",
        header=True,
        inferSchema=True,
    )
df.display()

# Transaction 01 - Write Data as Delta Format

In [0]:
df.write.format("delta").save(
    "/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta_new"
)

# Transaction 02 - Overwrite Delta Folder

In [0]:
from pyspark.sql.functions import col
df.filter(col("country")=="India").write.format("delta").save(
    "/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta_new",
    mode="overwrite"
)

# Reads the latest transaction

In [0]:
spark.read.load("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta_new").display()

# Maintains versions [Advantage]

In [0]:
spark.read.option("versionAsOf", 1).load(
    "/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta_new"
).display()

# Read Transaction Logs

## Appraoch 01

In [0]:
spark.read.load(format="text", path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta_new/part-00000-b255bd61-f524-45f6-9977-2bec576b05f0.c000.snappy.parquet").display()

In [0]:
spark.read.load(format="text", path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta_new/_delta_log/00000000000000000001.json").display()

## Approach 02

In [0]:
from delta.tables import DeltaTable
 
delta_table = DeltaTable.forPath(
    spark, "/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta_new/"
)
delta_table.history().display()

# Transaction 03 - Updates are allowed

In [0]:
df_delta = spark.read.load(
    "/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta_new/"
)
df_delta.createOrReplaceTempView("user_vw")

In [0]:
%sql
 
UPDATE user_vw
SET country='Bharat'
where country='India'